# Proyecto 1 NLP – Clasificación supervisada

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Procesamiento de Lenguaje Natural**   

## Integrantes: 
- Pablo Orellana
- Diego Leiva
- Renatto Guzmán

## Librerías

In [1]:
import pandas as pd
import re, unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SpanishStemmer
import nltk
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

## 1. Preprocesamiento del Corpus

### Cargar Corpus

Corpus obtenido de Kaggle en: [IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/datasets/luisdiegofv97/imdb-dataset-of-50k-movie-reviews-spanish)

In [2]:
df = pd.read_csv('Data/IMDB Dataset SPANISH.csv')
print(f"=== Dataset ===")
print(f"Documentos: {df.shape[0]}")
print(f"Características: {df.shape[1]}\n")
print("=== Información del Dataset ===")
df.info()
print("\n=== Primeras 5 filas del Dataset ===")
df.head()

=== Dataset ===
Documentos: 50000
Características: 5

=== Información del Dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   50000 non-null  int64 
 1   review_en    50000 non-null  object
 2   review_es    50000 non-null  object
 3   sentiment    50000 non-null  object
 4   sentimiento  50000 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB

=== Primeras 5 filas del Dataset ===


,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


### Procesamiento de DataFrame

In [3]:
# Eliminar columnas en ingles o innecesarias
df = df.drop(columns=['review_en', 'Unnamed: 0', 'sentiment'])

# Renombrar columnas
df = df.rename(columns={'review_es': 'review'})

# Convertir variable objetivo a binaria
df['sentimiento'] = df['sentimiento'].map({'positivo': 1, 'negativo': 0})

# Eliminar duplicados
df = df.drop_duplicates()

# Eliminar filas con valores nulos
df = df.dropna()

print(f"=== Dataset Procesado ===")
print(f"Documentos: {df.shape[0]}")
print(f"Características: {df.shape[1]}\n")
print("\n=== Primeras 5 filas del Dataset ===")
df.head()

=== Dataset Procesado ===
Documentos: 49599
Características: 2


=== Primeras 5 filas del Dataset ===


,review,sentimiento
0,Uno de los otros críticos ha mencionado que de...,1
1,Una pequeña pequeña producción.La técnica de f...,1
2,Pensé que esta era una manera maravillosa de p...,1
3,"Básicamente, hay una familia donde un niño peq...",0
4,"El ""amor en el tiempo"" de Petter Mattei es una...",1


### Normalización de Texto

In [4]:
def limpiar_texto(texto: str) -> str:
    """
    Función para limpiar y normalizar texto en español.
    Parámetros:
        texto (str): Texto a limpiar.
    Retorna:
        str: Texto limpio y normalizado.
    """
    # 1) a minúsculas
    s = str(texto).lower().strip()
    # 2) quitar tildes/diacríticos
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    # 3) remover URLs, emails, @usuarios, #hashtags
    s = re.sub(r'https?://\S+|www\.\S+', ' ', s)
    s = re.sub(r'\S+@\S+\.\S+', ' ', s)
    s = re.sub(r'[@#]\w+', ' ', s)
    # 4) quitar puntuación y símbolos
    s = re.sub(r'[^\w\s]', ' ', s, flags=re.UNICODE)  # elimina ¡¿, comas, puntos, comillas, etc.
    s = re.sub(r'_', ' ', s)
    # 5) espacios múltiples a uno
    s = re.sub(r'\s+', ' ', s)
    # 6) eliminar numeros
    s = re.sub(r'\d+', ' ', s)
    return s.strip()

In [5]:
# Crear columna normalizada
df["review_norm"] = df["review"].apply(limpiar_texto)

# Mostrar ejemplos de limpieza
print("Ejemplo (antes y después):\n")
for i in range(5):
    print(f"- Original: {df['review'].iloc[i][:50]}...")
    print(f"- Limpia  : {df['review_norm'].iloc[i][:50]}...\n")

# Verificar si hay vacios después de la limpieza
vacios = df['review_norm'].str.strip().eq('')
print(f"Número de filas con texto vacío después de la limpieza: {vacios.sum()}")
# Eliminar filas con texto vacío
df = df[~vacios]
print(f"Filas restantes después de eliminar textos vacíos: {df.shape[0]}")

Ejemplo (antes y después):

- Original: Uno de los otros críticos ha mencionado que despué...
- Limpia  : uno de los otros criticos ha mencionado que despue...

- Original: Una pequeña pequeña producción.La técnica de filma...
- Limpia  : una pequena pequena produccion la tecnica de filma...

- Original: Pensé que esta era una manera maravillosa de pasar...
- Limpia  : pense que esta era una manera maravillosa de pasar...

- Original: Básicamente, hay una familia donde un niño pequeño...
- Limpia  : basicamente hay una familia donde un nino pequeno ...

- Original: El "amor en el tiempo" de Petter Mattei es una pel...
- Limpia  : el amor en el tiempo de petter mattei es una pelic...

Número de filas con texto vacío después de la limpieza: 0
Filas restantes después de eliminar textos vacíos: 49599


### Stopwords
Se amplian las stopwords de NTLK con el conjunto de stopwords de [alir3z4 Stop Words](https://alir3z4.github.io/stop-words/)

In [6]:
def normalize(s: str) -> str:
    """
    Normaliza una cadena de texto, convirtiéndola a minúsculas, eliminando tildes y espacios en blanco.
    Parámetros:
        s (str): Cadena de texto a normalizar.
    Retorna:
        str: Cadena de texto normalizada.
    """
    s = s.lower().strip()
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_set(words):
    """
    Normaliza un conjunto de palabras.
    Parámetros:
        words (iterable): Conjunto de palabras a normalizar.
    Retorna:
        set: Conjunto de palabras normalizadas.
    """
    return {normalize(w) for w in words if w.strip()}

In [ ]:
# Descargar stopwords de NLTK si no se han descargado
nltk.download('stopwords', quiet=True)

# 1) base y extras
sw_base = set(stopwords.words('spanish'))
with open("Data/spanish_stopwords.txt", encoding="utf-8") as f:
    sw_extra = {line.strip() for line in f if line.strip()}

# 2) protegidas
negaciones = {
    "no","nunca","jamas","ni","sin","tampoco","ningun",
    "ninguna","ninguno","ningunas","ningunos","nadie","nada"
    }
contrastivos = {
    "pero","aunque","sin","embargo","sinembargo","no",
    "obstante","sino","excepto","salvo","aunquesi"
    }
intensif = {"muy","tan","tanto","tantos","tantas",
            "demasiado","demasiada","demasiados",
            "demasiadas","super","re","hiper","bastante",
            "apenas","poco","poca","pocos","pocas","casi",
            "algo","sumamente"
            }

protegidas = negaciones | contrastivos | intensif

# 3) normalizar todo
sw_total_norm = norm_set(sw_base | sw_extra) - norm_set(protegidas)
print("Stopwords finales:", len(sw_total_norm))

# 4) regex con bordes de palabra
pattern = re.compile(r'\b(?:' + '|'.join(sorted(map(re.escape, sw_total_norm), key=len, reverse=True)) + r')\b')

# 5) aplicar a la columna ya normalizada
df["review_sin_sw"] = (
    df["review_norm"]
      .str.replace(pattern, " ", regex=True)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)

# Chequeo rápido
print("\nEjemplos (antes y sin stopwords):\n")
for i in range(5):
    print(f"- Reseña        : {df['review_norm'].iloc[i][:50]}...")
    print(f"- Sin Stopwords : {df['review_sin_sw'].iloc[i][:50]}...\n")

# Verificar si hay vacios después de remover stopwords
vacios_sw = df['review_sin_sw'].str.strip().eq('')
print(f"Número de filas con texto vacío después de remover stopwords: {vacios_sw.sum()}")
# Eliminar filas con texto vacío
df = df[~vacios_sw]
print(f"Filas restantes después de eliminar textos vacíos: {df.shape[0]}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Stopwords finales: 564

Ejemplos (antes y sin stopwords):

- Reseña        : uno de los otros criticos ha mencionado que despue...
- Sin Stopwords : criticos mencionado oz episodio enganchado razon e...

- Reseña        : una pequena pequena produccion la tecnica de filma...
- Sin Stopwords : pequena pequena produccion tecnica filmacion muy i...

- Reseña        : pense que esta era una manera maravillosa de pasar...
- Sin Stopwords : pense maravillosa pasar semana verano demasiado ca...

- Reseña        : basicamente hay una familia donde un nino pequeno ...
- Sin Stopwords : basicamente familia nino pequeno jake piensa zombi...

- Reseña        : el amor en el tiempo de petter mattei es una pelic...
- Sin Stopwords : amor petter mattei pelicula visualmente impresiona...

Número de filas con texto vacío después de remover stopwords: 0
Filas restantes después de eliminar textos vacíos: 49599


### Tokenización y Stemming

In [8]:
# Descargar recursos de NLTK si no se han descargado
nltk.download('punkt', quiet=True)

# Inicializar el stemmer en español
stemmer = SpanishStemmer()

# Aplicar stemming a la columna sin stopwords
# tokens crudos (sin stopwords)
df["tokens"] = df["review_sin_sw"].apply(lambda x: word_tokenize(x, language="spanish"))

# tokens stem (lista)
df["tokens_stem"] = df["tokens"].apply(lambda xs: [stemmer.stem(t) for t in xs])

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,review,sentimiento,review_norm,review_sin_sw,tokens,tokens_stem
0,Uno de los otros críticos ha mencionado que de...,1,uno de los otros criticos ha mencionado que de...,criticos mencionado oz episodio enganchado raz...,"[criticos, mencionado, oz, episodio, enganchad...","[critic, mencion, oz, episodi, enganch, razon,..."
1,Una pequeña pequeña producción.La técnica de f...,1,una pequena pequena produccion la tecnica de f...,pequena pequena produccion tecnica filmacion m...,"[pequena, pequena, produccion, tecnica, filmac...","[pequen, pequen, produccion, tecnic, filmacion..."
2,Pensé que esta era una manera maravillosa de p...,1,pense que esta era una manera maravillosa de p...,pense maravillosa pasar semana verano demasiad...,"[pense, maravillosa, pasar, semana, verano, de...","[pens, maravill, pas, seman, veran, demasi, ca..."
3,"Básicamente, hay una familia donde un niño peq...",0,basicamente hay una familia donde un nino pequ...,basicamente familia nino pequeno jake piensa z...,"[basicamente, familia, nino, pequeno, jake, pi...","[basic, famili, nin, pequen, jak, piens, zombi..."
4,"El ""amor en el tiempo"" de Petter Mattei es una...",1,el amor en el tiempo de petter mattei es una p...,amor petter mattei pelicula visualmente impres...,"[amor, petter, mattei, pelicula, visualmente, ...","[amor, pett, mattei, pelicul, visual, impresio..."


### Levenshtein
No se aplica a todo el dataset debido a que:
- El corpus tiene ~50 000 reseñas → millones de palabras únicas.
- Comparar cada palabra con todas las demás sería $(O(n^2))$, ineficiente y sin valor práctico.

In [9]:
def levenshtein(a: str, b: str) -> int:
    """
    Distancia de edición mínima entre a y b.
    
    Parámetros:
        a (str): Primera cadena.
        b (str): Segunda cadena.
    Retorna:
        int: Distancia de edición mínima.
        """
    if a == b:
        return 0
    if not a:
        return len(b)
    if not b:
        return len(a)
    dp = np.zeros((len(a)+1, len(b)+1), dtype=int)
    dp[0, :] = np.arange(len(b)+1)
    dp[:, 0] = np.arange(len(a)+1)
    for i in range(1, len(a)+1):
        for j in range(1, len(b)+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i, j] = min(dp[i-1, j]+1, dp[i, j-1]+1, dp[i-1, j-1]+cost)
    return dp[len(a), len(b)]

In [10]:
# Construcción del vocabulario
# Tomamos los tokens sin stem para detectar errores reales
vocab_counts = Counter(t for toks in df["tokens"] for t in toks)
print(f"Vocabulario total: {len(vocab_counts)} palabras")

# Palabras frecuentes (consideradas válidas)
freq_words = {w for w, c in vocab_counts.items() if c >= 5}

# Ejemplo de corrección
palabra = "exelente"
distancias = {w: levenshtein(palabra, w) for w in list(freq_words)[:2000]}  # comparar con un subconjunto
sugerencia = min(distancias, key=distancias.get)
print(f"Palabra: {palabra} → Sugerencia más cercana: {sugerencia}")

# Opcional: aplicar a todo el vocabulario raro (<3 apariciones)
rare = {w for w, c in vocab_counts.items() if c < 3}
correcciones = {}
for w in list(rare)[:20]:
    candidatos = {v: levenshtein(w, v) for v in freq_words if abs(len(v)-len(w)) <= 2}
    if candidatos:
        best = min(candidatos, key=candidatos.get)
        if candidatos[best] <= 1:
            correcciones[w] = best

print("\nEjemplos de correcciones detectadas:")
for k, v in list(correcciones.items())[:10]:
    print(f"{k} → {v}")


Vocabulario total: 169469 palabras
Palabra: exelente → Sugerencia más cercana: entiende

Ejemplos de correcciones detectadas:
negociable → negociables
pilotada → pilotado
sidewalks → sidewalk
chateas → chatear
isenti → senti
gaseados → gaseado
brosnon → brosnan
carano → parano
infidel → infiel
bragging → dragging
